In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5,6,7'

In [2]:
import torch
import torch.nn as nn

import transformers

from transformers import (
    Qwen2ForCausalLM, 
    Qwen2TokenizerFast,
    AutoTokenizer,
    AutoConfig,
    AutoModel,
    pipeline
)

/mnt/nv1/yeqiang/lcx/nicy/envs/VideoReward/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
config = AutoConfig.from_pretrained(
    '/mnt/nv1/yeqiang/lcx/nicy/.cache/huggingface/hub/models--Qwen--Qwen2-1.5B-Instruct'
)
config.output_dim =1
config

Qwen2Config {
  "_name_or_path": "/mnt/nv1/yeqiang/lcx/nicy/.cache/huggingface/hub/models--Qwen--Qwen2-1.5B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 1536,
  "initializer_range": 0.02,
  "intermediate_size": 8960,
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 12,
  "num_hidden_layers": 28,
  "num_key_value_heads": 2,
  "output_dim": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": 32768,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.49.0",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

In [4]:
tokenizer = AutoTokenizer.from_pretrained(
    '/mnt/nv1/yeqiang/lcx/nicy/.cache/huggingface/hub/models--Qwen--Qwen2-1.5B-Instruct',
    padding_side="right"
)
isinstance(tokenizer, Qwen2TokenizerFast)

True

In [5]:
tokenizer.add_special_tokens(
    {'additional_special_tokens':["|<fkjalkf>|"]})
special_token_ids = tokenizer.convert_tokens_to_ids(["|<fkjalkf>|"])

In [18]:
from transformers import Qwen2VLForConditionalGeneration
model = Qwen2VLForConditionalGeneration.from_pretrained(
    '/mnt/nv1/yeqiang/lcx/nicy/.cache/huggingface/hub/models--Qwen--Qwen2-VL-2B-Instruct/snapshots/895c3a49bc3fa70a340399125c650a463535e71c'
)
model

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.09it/s]


Qwen2VLForConditionalGeneration(
  (visual): Qwen2VisionTransformerPretrainedModel(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
    )
    (rotary_pos_emb): VisionRotaryEmbedding()
    (blocks): ModuleList(
      (0-31): 32 x Qwen2VLVisionBlock(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): VisionSdpaAttention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (mlp): VisionMlp(
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (act): QuickGELUActivation()
          (fc2): Linear(in_features=5120, out_features=1280, bias=True)
        )
      )
    )
    (merger): PatchMerger(
      (ln_q): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (mlp): Seq

In [17]:
print([name for name, _ in model.named_parameters()])

['embed_tokens.weight', 'layers.0.self_attn.q_proj.weight', 'layers.0.self_attn.q_proj.bias', 'layers.0.self_attn.k_proj.weight', 'layers.0.self_attn.k_proj.bias', 'layers.0.self_attn.v_proj.weight', 'layers.0.self_attn.v_proj.bias', 'layers.0.self_attn.o_proj.weight', 'layers.0.mlp.gate_proj.weight', 'layers.0.mlp.up_proj.weight', 'layers.0.mlp.down_proj.weight', 'layers.0.input_layernorm.weight', 'layers.0.post_attention_layernorm.weight', 'layers.1.self_attn.q_proj.weight', 'layers.1.self_attn.q_proj.bias', 'layers.1.self_attn.k_proj.weight', 'layers.1.self_attn.k_proj.bias', 'layers.1.self_attn.v_proj.weight', 'layers.1.self_attn.v_proj.bias', 'layers.1.self_attn.o_proj.weight', 'layers.1.mlp.gate_proj.weight', 'layers.1.mlp.up_proj.weight', 'layers.1.mlp.down_proj.weight', 'layers.1.input_layernorm.weight', 'layers.1.post_attention_layernorm.weight', 'layers.2.self_attn.q_proj.weight', 'layers.2.self_attn.q_proj.bias', 'layers.2.self_attn.k_proj.weight', 'layers.2.self_attn.k_proj

In [7]:
model = Qwen2ForCausalLM.from_pretrained(
        '/mnt/nv1/yeqiang/lcx/nicy/.cache/huggingface/hub/models--Qwen--Qwen2-1.5B-Instruct',
        local_files_only=True, # 只使用本地文件，不尝试下载
        torch_dtype=torch.float16, # 覆盖默认的torch.dtype并以指定的数据类型加载模型
        device_map='cuda:0'
)
model.lm_head

Linear(in_features=1536, out_features=151936, bias=False)

In [8]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [9]:
print(model.device)

cuda:0


In [10]:
import torch
a = torch.Tensor([1,1,1,0,0])
b = torch.Tensor([0,0,0,1,1])

In [11]:
print(torch.eq(a, b).int())

tensor([0, 0, 0, 0, 0], dtype=torch.int32)


In [12]:
a != torch.Tensor([1])

tensor([False, False, False,  True,  True])

In [20]:
from datasets import load_dataset

dataset = load_dataset(
    'json',
    data_files='/mnt/nv1/yeqiang/lcx/nicy/workspace/code_generation/AlignLLM4Code_GRPO/raw_data/75k/lcs_split_data/functionality_lcs_split_results.jsonl'
)


Generating train split: 494 examples [00:00, 20954.76 examples/s]


In [23]:
dataset['train']

Dataset({
    features: ['index', 'programming_language', 'code-instruction', 'generation_model', 'answer', 'final_score'],
    num_rows: 494
})

In [29]:
a = torch.Tensor([[6,6,6,6]])
nn.MSELoss()(a,a)

tensor(0.)